In [7]:
# Code from Chapter 4 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)
 
# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.
 
# Stephen Marsland, 2008, 2014
 
import numpy as np
import matplotlib.pyplot as plt
class mlp:
    """ A Multi-Layer Perceptron"""
     
    def __init__(self,inputs,targets,nhidden,beta=1):
        """ Constructor """
        # Set up network size
        self.nin = np.shape(inputs)[1] #입력 데이터의 종류
        self.nout = np.shape(targets)[1] #목표값의 종류
        self.ndata = np.shape(inputs)[0] #입력 데이터의 개수
        self.nhidden = nhidden
 
        self.beta = beta
     
        # Initialise network
        #(nin+1, nhidden)크기의 배열을 가지며 -1과1 사이의 랜덤한 값을 입력값의 종류를 루트로 씌운 값으로 초기화
        self.weights1 = (np.random.rand(self.nin+1,self.nhidden)-0.5)*2/np.sqrt(self.nin)
        #(nhidden+1, nout)크기의 배열을 가지며 이번에는 은닉층의 개수를 루트로 씌움
        self.weights2 = (np.random.rand(self.nhidden+1,self.nout)-0.5)*2/np.sqrt(self.nhidden)
       
    def mlptrain(self,inputs,targets,eta,niterations):
        """ Train the thing """   
        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs,-np.ones((self.ndata,1))),axis=1)
        change = range(self.ndata)
     
        updatew1 = np.zeros((np.shape(self.weights1)))#weight1 크기랑 같은 배열을 0으로 초기화
        updatew2 = np.zeros((np.shape(self.weights2)))#weight2 크기랑 같은 배열을 0으로 초기화
             
        for n in range(niterations):
     
            self.outputs = self.predict(inputs) #결과는 전향의 결과
 
            error = 0.5*np.sum((self.outputs-targets)**2) #에러제곱합 함수
            if (np.mod(n,100)==0): #100의 배수마다 출력
                print("Iteration: ",n, " Error: ",error)   
 
            deltak = self.beta*(self.outputs-targets)*self.outputs*(1.0-self.outputs)

            deltaj = self.hidden*self.beta*(1.0-self.hidden)*(np.dot(deltak,np.transpose(self.weights2)))
   
            updatew1 = eta*(np.dot(np.transpose(inputs),deltaj[:,:-1]))
            updatew2 = eta*(np.dot(np.transpose(self.hidden),deltak))
            self.weights1 -= updatew1
            self.weights2 -= updatew2
                 
            # Randomise order of inputs (not necessary for matrix-based calculation)
            #np.random.shuffle(change)
            #inputs = inputs[change,:]
            #targets = targets[change,:]
             
    def predict(self,inputs): #전향
        """ Run the network forward """
       
        self.hidden = np.dot(inputs,self.weights1); #입력과 weight1의 곱
        self.hidden = 1.0/(1.0+np.exp(-self.beta*self.hidden)) #시그모이드 활성화 함수를 적용
        self.hidden = np.concatenate((self.hidden,-np.ones((np.shape(inputs)[0],1))),axis=1) #-1값을 갖는 바이어스 노드 추가
 
        outputs = np.dot(self.hidden,self.weights2); #은닉층의 결과값과 weight2의 곱
 
        return 1.0/(1.0+np.exp(-self.beta*outputs))
        
    def confmat(self,inputs,targets):
        """Confusion matrix"""
 
        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs,-np.ones((np.shape(inputs)[0],1))),axis=1)
        outputs = self.predict(inputs)
         
        nclasses = np.shape(targets)[1]
 
        if nclasses==1: #타겟의 종류가 1개일 때
            nclasses = 2
            outputs = np.where(outputs>0.5,1,0) #이진시그모이드 함수니까 0.5를 기준으로한다.
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs,1)
            targets = np.argmax(targets,1)
 
        cm = np.zeros((nclasses,nclasses))
        for i in range(nclasses):
            for j in range(nclasses):
                cm[i,j] = np.sum(np.where(outputs==i,1,0)*np.where(targets==j,1,0))
 
        print("Confusion matrix is:")
        print(cm)
        print("Percentage Correct: ",np.trace(cm)/np.sum(cm)*100)

In [9]:
import numpy as np
from mlxtend.plotting import plot_decision_regions

anddata = np.array([[0,0,0],[0,1,0],[1,0,0],[1,1,1]])
xordata = np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]])
 
p = mlp(anddata[:,0:2],anddata[:,2:3],2)
p.mlptrain(anddata[:,0:2],anddata[:,2:3],0.25,1001)
p.confmat(anddata[:,0:2],anddata[:,2:3])

q = mlp(xordata[:,0:2],xordata[:,2:3],2)
q.mlptrain(xordata[:,0:2],xordata[:,2:3],0.25,5001)
q.confmat(xordata[:,0:2],xordata[:,2:3])

Iteration:  0  Error:  0.6159448745048918
Iteration:  100  Error:  0.35327266438033655
Iteration:  200  Error:  0.3151072834759739
Iteration:  300  Error:  0.26330104250578856
Iteration:  400  Error:  0.20135393973859944
Iteration:  500  Error:  0.1397583979361719
Iteration:  600  Error:  0.09244435099793408
Iteration:  700  Error:  0.06190122251508156
Iteration:  800  Error:  0.04323439847996399
Iteration:  900  Error:  0.0316804797331187
Iteration:  1000  Error:  0.024241715654806387
Confusion matrix is:
[[3. 0.]
 [0. 1.]]
Percentage Correct:  100.0
Iteration:  0  Error:  0.5371030698933096
Iteration:  100  Error:  0.4972227533299547
Iteration:  200  Error:  0.4957731492652257
Iteration:  300  Error:  0.49325711103416026
Iteration:  400  Error:  0.48863141781438313
Iteration:  500  Error:  0.4797086451064974
Iteration:  600  Error:  0.46200355123365355
Iteration:  700  Error:  0.42687286187870066
Iteration:  800  Error:  0.3604062845757652
Iteration:  900  Error:  0.2597984331943956
